# Импорт библиотек

In [1]:
import numpy as np
from gzip import open # NB: overrides standard open()
import pickle as pkl
import pandas as pd
import warnings 
from glob import glob
import os
import math
import functools

### Отключим предупреждения Anaconda

In [2]:
warnings.simplefilter('ignore')

# Загрузим данные

In [3]:
Xdata_numpy = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/X.pkl.gz', 'rb'))
Ydata = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/y.pkl.gz', 'rb'))

# N-PLS1

## Выберем столбец у с которым будем работать

In [4]:
m=0

# Приближение к правильному алгоритму

In [104]:
from sklearn.model_selection import train_test_split

In [345]:
# a-numpy array;  b-pandas array
def error(a,b):
    s=0
    for i in range(0,len(a)):
        s+=((a[i]-b.iloc[i]))**2
    return s

In [368]:
parameters={'numbers_of_learning':3, 'numbers_of_components':15}

In [373]:
output=np.zeros([parameters['numbers_of_components']])
for v in range(0,parameters['numbers_of_learning']):
    X_train, X_test, y_train, y_test = train_test_split(
        Xdata_numpy["X"], Ydata.iloc[:,m], test_size=0.2857 
        )
    x=X_train
    y=y_train
    y=np.array(y)
    x=np.array(x)

    Tt=[]
    mass=np.zeros([y.shape[0]])
    for f in range(0,parameters['numbers_of_components']):
        z=np.zeros([x.shape[1],x.shape[2]])
        for i in range(0,x.shape[2]):
            for j in range(0,x.shape[1]):
                zsum=0
                for k in range(0,x.shape[0]):
                    zsum= zsum+ x[k,j,i]*y[k]
                z[j,i]=zsum
        Wk, S, WI = np.linalg.svd(z)
        w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
        w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
        
        t=[]
        for h in range(0,x.shape[0]):
            t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
        t=np.array(t).reshape(x.shape[0],1)
        Tt+=[t]
        T=np.array(Tt).reshape(x.shape[0],f+1)
        bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
        for g in range(0,x.shape[0]):
            x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
        y=y-(np.dot(T,bf))
        mass+=np.dot(T,bf)
        print(error(mass,y_train))
        output[f]+=error(mass,y_train)
    print()
np.array(output).reshape(1,parameters['numbers_of_components'])

4669.6385217742545
4229.882136803922
4174.888472906578
4090.6137039895084
3323.6287428820046
1725.5022563688376
1236.628731360824
1041.8830287165129
725.813147720635
356.14266987486883
218.33742807879068
112.69405044292813
53.74531337908869
13.818071580481444
10.796699206661671

5443.7735695966
4751.768954119363
4154.9817021547315
4065.7599283848945
3493.2606221163487
3275.5147585237287
2967.876156542172
1954.5317525739745
1468.2229561401275
902.3569238185453
582.0314144288868
364.9554776363416
267.0938111007083
54.929355228245896
17.91166588137039

6247.593182069793
5743.6453093542295
4822.316663020836
3724.7432511333895
3409.2220904575456
2847.8935149291283
1966.2306884063373
1531.66389396502
1247.2354102821953
1061.628808187996
752.6725471358013
584.9068503192746
431.58068749314003
157.19781443622912
63.80219893458045



array([[16361.00527344, 14725.29640028, 13152.18683808, 11881.11688351,
        10226.11145546,  7848.91052982,  6170.73557631,  4528.07867526,
         3441.27151414,  2320.12840188,  1553.04138964,  1062.5563784 ,
          752.41981197,   225.94524124,    92.51056402]])

In [372]:
output

array([5450.19419996, 4930.33342223, 4576.11980653, 4300.81569636,
       3855.76017733, 3140.43392713, 2660.8926247 , 2463.17024479,
       2171.27199806, 1292.97681881,  619.82549492,  309.35665498,
        137.82678773,   61.68889269,   53.60363593])

###### Массив mass собирает разложение обратно, и именно он важен для определения числа комронент

# Попробую оформить всё в виде полноценного класса

In [ ]:
class N_PLS1():
    def error(a,b):
        s=0
        for i in range(0,len(a)):
            s+=((a[i]-b.iloc[i]))**2
        return s
    def function_1(x,y,fn):
        Tt=[]
        mass=np.zeros([y.shape[0]])
        output=[]
        for f in range(0,fn):
            z=np.zeros([x.shape[1],x.shape[2]])
            for i in range(0,x.shape[2]):
                for j in range(0,x.shape[1]):
                    zsum=0
                    for k in range(0,x.shape[0]):
                        zsum= zsum+ x[k,j,i]*y[k]
                    z[j,i]=zsum
            Wk, S, WI = np.linalg.svd(z)
            w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
            w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
            t=[]
            for h in range(0,x.shape[0]):
                t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
            t=np.array(t).reshape(x.shape[0],1)
            Tt+=[t]
            T=np.array(Tt).reshape(x.shape[0],f+1)
            bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
            for g in range(0,x.shape[0]):
                x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
            y=y-(np.dot(T,bf))
            mass+=np.dot(T,bf)
            output+=[error(mass,y_train)]
        return output
    def fit(x,y,fn):
        
            
        return y

In [179]:
y

array([34.19308759, 35.72443085, 46.32211944, 35.69579348, 47.44184036,
       33.34204937, 42.44462856, 39.9018207 , 44.39973652, 54.64066407,
       53.71596802, 44.04933373, 36.11255278, 34.24246421, 36.22622795,
       26.1974747 , 41.0398005 , 31.41066637, 34.07047331, 42.14770231,
       45.44679116, 21.41118209, 44.54683093, 35.35460545, 36.47993911])